In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

In [3]:
import torch
from transformers import AutoConfig, AutoModel, GPT2Tokenizer, TextDataset, TrainingArguments
from transformers import DataCollatorForLanguageModeling, Trainer, AutoTokenizer, GPT2LMHeadModel

In [13]:
config = AutoConfig.from_pretrained("/content/drive/MyDrive/NLP/final_model")
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/NLP/final_model")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

## Dataset for testing

In [5]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/drive/MyDrive/NLP/t.csv',
    block_size=128
)

validation_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/MyDrive/NLP/v.csv",
    block_size=128
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


## Dataset for actual training

In [ ]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/MyDrive/NLP/dataset_1.csv",
    block_size=128
)

validation_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/MyDrive/NLP/validation.csv",
    block_size=128
)

test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/MyDrive/NLP/test.csv",
    block_size=128
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [6]:
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW

____

In [ ]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [ ]:
print(model.parameters())

<generator object Module.parameters at 0x7a8a7465b3e0>


In [ ]:
config.n_embd, config.vocab_size

(768, 50257)

Just in case- the tokenization process

In [ ]:
input_ids = []
maximum_sequence_length = config.n_embd

for text in train_dataset:
  tokens = tokenizer.encode(text, add_special_tokens=True, max_length=maximum_sequence_length, truncation=True)
  if len(tokens) < maximum_sequence_length:
    tokens = tokens + [tokenizer.pad_token_id] * (maximum_sequence_length - len(tokens))
  else:
    tokens = tokens[:maximum_sequence_length]

  input_ids.append(tokens)

input_ids = torch.tensor(input_ids)

In [14]:
# train_loader = DataLoader(train_dataset, batch_size=4 , shuffle=True)

device = ('cuda' if torch.cuda.is_available() else 'cpu')

class CustomModel(torch.nn.Module):
  def __init__(self, pretrained_model, config):
    super(CustomModel, self).__init__()
    self.transformer = pretrained_model
    self.config = config
    # self.lm_head = torch.nn.Linear(self.config.n_embd, self.config.vocab_size, bias=False)

    # Additional layers
    self.Linear1 = torch.nn.Linear(self.config.vocab_size, 512)
    self.Dropout1 = torch.nn.Dropout(0.1)
    self.Linear2 = torch.nn.Linear(512, self.config.n_embd)
    self.Dropout2 = torch.nn.Dropout(0.1)
    self.Linear3 = torch.nn.Linear(self.config.n_embd, self.config.vocab_size)

  def forward(self, input_ids, attention_mask=None, token_type_ids=None):
    outputs = self.transformer(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    hidden_states = outputs.logits

    hidden_states = self.Linear1(hidden_states)
    hidden_states = torch.nn.functional.gelu(hidden_states)
    hiddne_states = self.Dropout1(hidden_states)
    hidden_states = self.Linear2(hidden_states)
    hidden_states = torch.nn.functional.gelu(hidden_states)
    hidden_states = self.Dropout2(hidden_states)
    logits = self.Linear3(hidden_states)

    return logits

  def generate_text(self, input_ids, max_length=50):
    with torch.no_grad():
      generated_ids = []
      current_token = input_ids
      for _ in range(max_length):
        logits = self(current_token)
        predicted_token = torch.argmax(logits[:, -1, :], dim=-1)
        generated_ids.append(predicted_token.item())
        current_token = torch.cat((current_token, predicted_token.unsqueeze(1)), dim=1)
      return generated_ids


In [15]:
for param in model.parameters():
  param.requires_grad = False

In [16]:
model = CustomModel(model, config)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
model

CustomModel(
  (transformer): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  )
  (Linear1): Linear(in_feature

In [17]:
train_loader = DataLoader(train_dataset, batch_size=4 , shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=4, shuffle=True)

# Execute this cell only if you want to load a model

In [ ]:
checkpoint = torch.load('model_checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
total_loss = checkpoint['training_loss']
validation_loss = checkpoint['validation_loss']
step = checkpoint['steps']

I will have to write different training loop for training my model from a checkpoint

In [ ]:
%%time
from tqdm import tqdm

device = ('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

epochs=2
step = 0
model.train()
total_loss = 0.

for epoch in range(epochs):
  train_loader = tqdm(train_loader, total=len(train_loader))

  for batch in train_loader:
    input_ids, attention_mask, token_type_ids, targets = batch

    input_ids = input_ids.to(device)
    targets = targets.to(device)

    optimizer.zero_grad()
    outputs = model(input_ids)
    loss = criterion(outputs.view(-1, config.vocab_size), targets.view(-1))
    loss.backward()
    total_loss += loss.item()
    train_loader.set_description(f"Epoch: {epoch+1}")
    train_loader.set_postfix(loss=total_loss)
    step += 1
    if step % 400 == 0:
      print(f"Epoch: {epoch+1}, Loss: {total_loss/len(train_loader)}")

      # VALIDATION
      # model.eval()
      # validation_loss = 0.

      # with torch.no_grad():
      #   for batch in val_loader:
      #     input_ids, attention_mask, token_type_ids, targets = batch
      #     outputs = model(input_ids, attention_mask, token_type_ids)
      #     loss = criterion(outputs.view(-1, config.vocab_size), targets.view(-1))
      #     validation_loss += loss.item()
      #     break
      # average_validation_loss = validation_loss / len(val_loader)
      # print(f"Epoch: {epoch+1}, Validation Loss: {average_validation_loss}")

      # SAVE A CHECKPOINT
      torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'training_loss': total_loss,
          'validation_loss': validation_loss,
          'steps': step
      }, "/content/drive/MyDrive/NLP/trained_again_model/model_checkpoint.pth")

torch.save(model, '/content/drive/MyDrive/NLP/trained_again_model/model.pth')

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Test the model

In [ ]:
model = CustomModel(model, config)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


# Load your fine-tuned model checkpoint
checkpoint_path = '/content/drive/MyDrive/NLP/trained_again_model/model_checkpoint.pth'
checkpoint = torch.load(checkpoint_path, map_location='cpu')  # Make sure to specify the device you want to use

# Load the model's state_dict
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # Set the model to evaluation mode

# Define a function to chat with the model
def chat_with_model(prompt, max_length=50):
    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=max_length)
    with torch.no_grad():
        output = model.generate_text(input_ids, max_length=max_length, )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Start a conversation
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Chatbot: Goodbye!")
        break
    response = chat_with_model(user_input)
    print("Chatbot:", response)

RuntimeError: ignored